In [158]:
from app.database.DBInterface import DBInterface
from app.database.util import functions as f
import configs as cfg

import hashlib

In [159]:
def _replace_na(df, col, val=None):

    '''
    Replace NA values in a column with a value
        df: dataframe
        col: column name
        val: value to replace NA with
    Returns dataframe
    '''
    import numpy as np
    df[col] = df[col].fillna(val)
    df[col] = df[col].replace(np.nan, val)
    df[col] = df[col].replace('NULL', val)
    df[col] = df[col].replace('---', val)
    return df


def parse_total_time(df, tf_col='TimeFormat'):
    '''
    Parse the total time of a fight
        df: dataframe
        tf_col: column name of the time format
    Returns dataframe
    '''
    import numpy as np
    df['PossibleRounds'] = np.where(df[tf_col] == '5 Rnd (5-5-5-5-5)', 5, 3)
    df['PossibleTime'] = np.where(df[tf_col] == '5 Rnd (5-5-5-5-5)', 25 * 60, 15 * 60)
    df = df.drop([tf_col], axis=1)
    return df


def split_col(df, old_col, sep=' ', new_col=None, suffix=['LND', 'ATMPT'],  drop=True, func=None):
    '''
    Split a column into two columns based on a separator
        df: dataframe
        old_col: column name to split
        sep: separator
        new_col: new column name
        suffix: suffix for new column names
        drop: drop old column
        func: function to apply to new columns
    Returns dataframe
    '''
    split_col = df[old_col].str.split(sep, )
    new_cols = None
    if new_col is None:
        new_col = old_col
    if suffix is None:
        suffix = [i for i in range(len(split_col[0]))]
    new_cols = [f"{new_col}_{s}" for s in suffix]
    for i in range(len(new_cols)):
        df[new_cols[i]] = split_col.str[i]
    for col in df[new_cols]:
        if func is not None:
            df[col] = df[col].apply(func)
    if drop:
        df = df.drop([old_col], axis=1)
    return df


def convert_percent_columns(df):
    '''
    Convert columns with 'pct' in col name to float
        df: dataframe
    Returns dataframe
    '''
    percent_cols = [col for col in df.columns if 'pct' in col.lower()]
    for col in percent_cols:
        try:
            df[col] = df[col].str.replace('%', '').astype(float) / 100
        except AttributeError:
            pass
    return df


def convert_to_secs(time):
    '''
    Convert time of the format 'MM:SS' to seconds
        time: time string
    Returns seconds
    '''
    if time is None:
        return 0
    try:
        time = time.split(':')
        return int(time[0]) * 60 + int(time[1])
    except ValueError:
        return 0

    
def total_fight_time(row):
    '''
    Calculate the total fight time based on the round and time
        row: row of dataframe
    Returns total fight time in seconds
    '''
    if row['Time'] == '5:00' and int(row['Round'])==row['PossibleRounds']:
        return row['PossibleTime']
    elif row['Round'] == '1':
        return convert_to_secs(row['Time'])
    else:
        rnds_done = int(row['Round']) - 1
        return (rnds_done * 5 * 60) + convert_to_secs(row['Time'])



def convert_to_int(df, cols):
    '''
    Convert columns to int
        df: dataframe
        cols: list of columns to convert
    Returns dataframe
    '''
    for col in cols:
        df[col] = df[col].astype(int)
    return df


def replace_na(df):
    '''
    replace NA values in dataframe with 0 for numeric columns and '' for string columns
        df: dataframe
    Returns dataframe
    '''
    dtypes = ['int32', 'float64', 'int64']
    for col in df.columns:
        if df[col].dtype in dtypes:
            df[col] = df[col].fillna(0)
        else:
            df[col] = df[col].fillna('')
    return df


def rename_cols(df, old_cols, new_cols):
    '''
    Rename columns in dataframe
        df: dataframe
        old_cols: list of old column names or regex
        new_cols: list of new column names
    Returns dataframe
    '''
    import re
    df.columns = [re.sub(old_cols, new_cols, col) for col in df.columns]
    return df


def convert_to_float(df, cols):
    '''
    Convert columns to float
        df: dataframe
        cols: list of columns to convert
    Returns dataframe
    '''
    for col in cols:
        df[col] = df[col].astype(float)
    return df


def convert_to_inches(value):
    '''
    Convert height to inches
        value: height string
    Returns height in inches
    '''
    import re
    if value is None:
        return 0
    try:
        value = re.findall(r'\d+', value)
        return int(value[0]) * 12 + int(value[1])
    except ValueError:
        return 0


def prioritize_columns(df, col_list):
    '''
    Move columns to the front of the dataframe
        df: dataframe
        col_list: list of columns to move to the front
    Returns dataframe
    '''
    other_cols = [col for col in df.columns if col not in col_list]
    return df[col_list + other_cols]

def reorder_list(lst, prio_cols, deprio=False):
    '''
    Similar to prioritize_columns, but for lists
        lst: list
        prio_cols: list of columns to move to the front
        deprio: reverse the list
    Returns list
    '''
    new_col_list = [col for col in lst if col in prio_cols] + [col for col in lst if col not in prio_cols]
    print(new_col_list)
    if deprio:
        return new_col_list[::-1]
    return new_col_list


def convert_to_bool(df, cols):
    '''
    Convert columns to boolean type
        df, dataframe
        cols: list of columns to convert
    Returns dataframe
    '''
    for col in cols:
        df[col] = df[col].astype(bool)
    return df

def hashify(df, cols, new_col_name = 'hashed'):
    '''
    Hashify columns in dataframe and create new column with hashed value
        df: dataframe
        cols: list of columns to hash
        new_col_name: name of new column
    Returns dataframe
    '''
    df[new_col_name] = df[cols].apply(lambda x: ''.join(x.dropna().astype(str)), axis=1)
    df[new_col_name] = df[new_col_name].apply(lambda x: hashlib.sha256(str(x).encode('utf-8')).hexdigest())
    return df


def separate_fighters(df):
    '''
    Separate fighters into two separate dataframes, one for each fighter, then merge them back together with the same columns
        df: dataframe
    Returns dataframe
    '''
    import pandas as pd
    f1_cols = [col for col in df.columns if 'f2' not in col.lower()] + ['F2', 'F2_NICKNAME']
    f1_df = df[f1_cols].rename({'F2': 'OPP_NAME', 'F2_NICKNAME':'OPP_NICKNAME'}, axis=1)
    f1_df.columns = [col.replace('F1','F') if 'F1' in col else col for col in f1_df.columns]

    f2_cols = [col for col in df.columns if 'f1' not in col.lower()] + ['F1', 'F1_NICKNAME']
    f2_df = df[f2_cols].rename({'F1': 'OPP_NAME', 'F1_NICKNAME':'OPP_NICKNAME'}, axis=1)
    f2_df.columns = [col.replace('F2','F') if 'F2' in col else col for col in f2_df.columns]

    return pd.concat([f1_df, f2_df], axis=0)

def ft(df, col, val):
    return df[df[col] == val]

def rms(x):
    import numpy as np
    rms = np.sqrt(np.mean(x**2))
    return rms

In [160]:
db = DBInterface(cfg.DB_PATH / 'ufc_raw')
bouts_df = db.Pdf('fights')
db.close()



bouts_df = hashify(bouts_df, ['Fighter1', 'Fighter2', 'EventName'], 'FightHash')

bouts_df = bouts_df.loc[bouts_df['TimeFormat'].isin(['3 Rnd (5-5-5)', '5 Rnd (5-5-5-5-5)'])].reset_index(drop=True)

bouts_df = bouts_df.replace('-{2,}', None, regex=True)
bouts_df = bouts_df.replace(r'"', '', regex=True)

bouts_df['Judge1_Score'] = bouts_df['Judge1_Score'].fillna(0)
bouts_df['Judge2_Score'] = bouts_df['Judge2_Score'].fillna(0)
bouts_df['Judge3_Score'] = bouts_df['Judge3_Score'].fillna(0)

bouts_df = parse_total_time(bouts_df)
bouts_df = split_col(bouts_df, 'Judge1_Score', ' - ', 'Judge1', ['F1Score', 'F2Score'], func=float)
bouts_df = split_col(bouts_df, 'Judge2_Score', ' - ', 'Judge2', ['F1Score', 'F2Score'], func=float)
bouts_df = split_col(bouts_df, 'Judge3_Score', ' - ', 'Judge3', ['F1Score', 'F2Score'], func=float)

of_columns = ['SIG_STR', 'TOTAL_STR', 'TD', 'HEAD', 'BODY', 'LEG', 'DISTANCE', 'CLINCH', 'GROUND']
for col in of_columns:
    f1, f2 = f'Fighter1_{col}', f'Fighter2_{col}'
    bouts_df = split_col(bouts_df, f1, ' of ', f'F1_{col}', func=int)
    bouts_df = split_col(bouts_df, f2, ' of ', f'F2_{col}', func=int)

bouts_df = convert_percent_columns(bouts_df)

bouts_df['Fighter1_CTRL'] = bouts_df['Fighter1_CTRL'].apply(convert_to_secs)
bouts_df['Fighter2_CTRL'] = bouts_df['Fighter2_CTRL'].apply(convert_to_secs)

bouts_df['Duration'] = bouts_df.apply(total_fight_time, axis=1)

bouts_df = convert_to_int(bouts_df, ['Fighter1_KD', 'Fighter2_KD', 'Fighter1_SUB_ATT', 'Fighter2_SUB_ATT', 'Fighter1_REV', 'Fighter2_REV'])

bouts_df = replace_na(bouts_df)

bouts_df.columns = [col.upper() for col in bouts_df.columns]
bouts_df = rename_cols(bouts_df, 'FIGHTER', 'F')

bouts_df = separate_fighters(bouts_df).reset_index(drop=True)

prio_cols = ['F', 'OPP_NAME']
bouts_df = prioritize_columns(bouts_df, prio_cols)

bouts_df = convert_to_bool(bouts_df, ['PERF_BONUS', 'FIGHT_BONUS', 'SUB_BONUS', 'KO_BONUS'])



bouts_df['WINNER'] = bouts_df['WINNER'] == bouts_df['F']
bouts_df['WINNER'] 

bouts_df = prioritize_columns(bouts_df, ['F', 'WINNER', 'OPP_NAME', 'WEIGHT_CLASS', 'METHOD', 'ROUND', 'DURATION', ])

bouts_df['SUBMISSION'] = (bouts_df['METHOD'] == 'Submission').astype(int)

bouts_df

,F,WINNER,OPP_NAME,WEIGHT_CLASS,METHOD,ROUND,DURATION,EVENTNAME,TIME,REFEREE,...,F_LEG_LND,F_LEG_ATMPT,F_DISTANCE_LND,F_DISTANCE_ATMPT,F_CLINCH_LND,F_CLINCH_ATMPT,F_GROUND_LND,F_GROUND_ATMPT,OPP_NICKNAME,SUBMISSION
0,TJ Brown,True,Erik Silva,Featherweight Bout,Submission,3,821,UFC 282: Blachowicz vs. Ankalaev,3:41,Mark Smith,...,0,0,13,25,6,7,5,6,The King,1
1,Cameron Saaiman,True,Steven Koslow,Bantamweight Bout,KO/TKO,3,853,UFC 282: Blachowicz vs. Ankalaev,4:13,Chris Tognoni,...,6,6,23,50,9,11,21,28,Obi Won Shinobi The Pillow,0
2,Chris Curtis,True,Joaquin Buckley,Middleweight Bout,KO/TKO,2,469,UFC 282: Blachowicz vs. Ankalaev,2:49,Jason Herzog,...,8,10,29,78,1,1,6,8,New Mansa,0
3,Billy Quarantillo,True,Alexander Hernandez,Featherweight Bout,KO/TKO,2,570,UFC 282: Blachowicz vs. Ankalaev,4:30,Chris Tognoni,...,9,10,52,109,21,22,14,20,The Great Ape,0
4,Edmen Shahbazyan,True,Dalcha Lungiambula,Middleweight Bout,KO/TKO,2,581,UFC 282: Blachowicz vs. Ankalaev,4:41,Chris Tognoni,...,3,4,37,72,7,10,10,25,Champion,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13387,Arnold Allen,True,Gilbert Melendez,Featherweight Bout,Decision - Unanimous,3,900,UFC 239: Jones vs. Santos,5:00,Marc Goddard,...,14,19,102,190,0,0,0,0,El Nino,0
13388,Michael Chiesa,True,Diego Sanchez,Welterweight Bout,Decision - Unanimous,3,900,UFC 239: Jones vs. Santos,5:00,Mark Smith,...,0,0,4,7,5,8,38,52,Nightmare,0
13389,Ben Askren,False,Jorge Masvidal,Welterweight Bout,KO/TKO,1,5,UFC 239: Jones vs. Santos,0:05,Jason Herzog,...,0,0,0,0,0,0,0,0,Gamebred,0
13390,Holly Holm,False,Amanda Nunes,UFC Women's Bantamweight Title Bout,KO/TKO,1,250,UFC 239: Jones vs. Santos,4:10,Marc Goddard,...,8,9,11,14,0,0,0,0,The Lioness,0


In [182]:
import re

db = DBInterface(cfg.DB_PATH / 'ufc_raw')
combatants_df = db.Pdf("fighters")
db.close()

combatants_df = combatants_df.replace("-{2,}", None, regex=True)


parsed_record = (
    combatants_df["Record"].str.findall(r"(\d+)-(\d+)-(\d+)").apply(lambda x: x[0])
)
combatants_df["WINS"] = parsed_record.apply(lambda x: x[0]).astype(int)
combatants_df["LOSSES"] = parsed_record.apply(lambda x: x[1]).astype(int)
combatants_df["DRAWS"] = parsed_record.apply(lambda x: x[2]).astype(int)
combatants_df.drop("Record", axis=1, inplace=True)

combatants_df["HEIGHT"] = combatants_df["Height"].apply(convert_to_inches)
combatants_df.drop("Height", axis=1, inplace=True)
combatants_df
combatants_df["Listed_Weight"] = (
    combatants_df["Listed_Weight"].str.extract(r"(\d+)").astype(float)
)
combatants_df["Reach"] = combatants_df["Reach"].str.extract(r"(\d+)").astype(float)

percent_cols = ["SSAcc", "SSDef", "TDAcc", "TDDef"]
for col in percent_cols:
    combatants_df[col] = combatants_df[col].str.extract(r"(\d+)").astype(float) / 100

combatants_df = rename_cols(combatants_df, 'Fighter', 'F')

float_cols = ["SSLpM", "SSApM", "TDavg", "SubAvg"]
combatants_df = convert_to_float(combatants_df, float_cols)

combatants_df.columns = [col.upper() for col in combatants_df.columns]

combatants_df = replace_na(combatants_df)
combatants_df.replace('', None, inplace=True)
combatants_df


combatants_df = prioritize_columns(
    combatants_df,
    [
        "F_NAME",
        "NICKNAME",
        "DOB",
        "HEIGHT",
        "WINS",
        "LOSSES",
        "DRAWS",
        "STANCE",
        "LISTED_WEIGHT",
        "REACH",
    ],
)
combatants_df

,F_NAME,NICKNAME,DOB,HEIGHT,WINS,LOSSES,DRAWS,STANCE,LISTED_WEIGHT,REACH,SSLPM,SSACC,SSAPM,SSDEF,TDAVG,TDACC,TDDEF,SUBAVG
0,Tom Aaron,None,"Jul 13, 1978",0,5,3,0,None,155.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0
1,Amir Aliakbari,None,"Jun 10, 1984",75,10,1,0,None,250.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0
2,Sultan Aliev,None,"Sep 17, 1984",71,15,3,0,Orthodox,170.0,74.0,1.41,0.45,2.20,0.56,1.76,0.38,1.00,0.0
3,Bill Algeo,Senor Perfecto,"Jun 09, 1989",72,16,7,0,Switch,145.0,73.0,5.68,0.53,4.14,0.46,0.77,0.38,0.55,0.0
4,Royce Alger,None,None,70,3,2,0,Orthodox,199.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3910,Ivan Valenzuela,Bam Bam,"Nov 17, 1992",74,8,2,0,Orthodox,185.0,77.0,0.00,0.00,9.60,0.33,0.00,0.00,0.00,0.0
3911,Timur Valiev,Lucky,"Jan 19, 1990",66,18,3,0,Orthodox,135.0,67.0,5.19,0.53,3.33,0.45,1.73,0.40,0.55,0.3
3912,Charlie Valencia,None,"Oct 31, 1974",63,12,7,0,Orthodox,135.0,64.0,2.58,0.37,2.61,0.77,1.40,0.53,0.75,0.5
3913,Egidijus Valavicius,None,None,72,29,12,0,Orthodox,200.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0


In [183]:
import numpy as np
db = DBInterface(cfg.DB_PATH / 'ufc_raw')
ufc_fighters = db.Pdf("officialufcfighters")
db.close()

ufc_fighters = ufc_fighters.replace("-{2,}", None, regex=True)
ufc_fighters = ufc_fighters.replace(r'"', '', regex=True)
ufc_fighters['Standing_Strikes'] = ufc_fighters['Standing_Strikes'].str.findall(r'(\d+)(?!\d* ?%)').apply(lambda x: int(x[0]) if x is not None else None)
ufc_fighters['Clinch_Strikes'] = ufc_fighters['Clinch_Strikes'].str.findall(r'(\d+)(?!\d* ?%)').apply(lambda x: int(x[0]) if x is not None else None)
ufc_fighters['Ground_Strikes'] = ufc_fighters['Ground_Strikes'].str.findall(r'(\d+)(?!\d* ?%)').apply(lambda x: int(x[0]) if x is not None else None)
ufc_fighters['Gender'] = np.where(ufc_fighters['Division'].str.lower().str.contains('women'), 'F', 'M')
ufc_fighters['Division_Rank'] = ufc_fighters['Division'].str.extract('(\d+)')
ufc_fighters['Division'] = ufc_fighters['Division'].str.findall(r"(Women's ?(\w+) (?<!Division)").apply(lambda x: x[-1] if x is not None else None)
ufc_fighters['PFP_Rank'] = ufc_fighters['PFP_Rank'].str.extract('(\d+)')
ufc_fighters.columns = ufc_fighters.columns.str.upper()
ufc_fighters = replace_na(ufc_fighters)
ufc_fighters;

In [184]:
bouts_df = hashify(bouts_df, ['F', 'F_NICKNAME'], 'FIGHTER_ID')
bouts_df = hashify(bouts_df, ['OPP_NAME', 'OPP_NICKNAME'], 'OPP_ID')
combatants_df = hashify(combatants_df, ['F_NAME', 'NICKNAME'], 'FIGHTER_ID')
ufcfighters_df = hashify(ufc_fighters, ['FIGHTER_NAME', 'NICKNAME'], 'FIGHTER_ID')

combatants_df = replace_na(ufc_fighters.merge(combatants_df, how='inner', on='FIGHTER_ID')).replace('', None)

In [185]:

db = DBInterface(cfg.DB_PATH / 'ufc_silver')
db.to_table(combatants_df, 'combatants')
db.to_table(bouts_df, 'bouts_df')
db.close()

In [186]:
import pandas as pd
_ = bouts_df['WEIGHT_CLASS'].str.lower().str.extract(r"(light |open |catch )?(\w*?weight)").fillna('')
bouts_df['WEIGHT_CLASS'] = _[0] + _[1]
map_dict = bouts_df[['FIGHTER_ID', 'WEIGHT_CLASS']]
map_dict = map_dict.groupby('FIGHTER_ID', as_index=True).max()
combatants_df['WEIGHT_CLASS'] = combatants_df['FIGHTER_ID'].map(map_dict['WEIGHT_CLASS'])

In [317]:
_ = bouts_df.groupby('FIGHTER_ID')[['SUBMISSION', 'F_SUB_ATT']].sum()
_['SUBMISSION_ACC'] = (_['SUBMISSION'] / _['F_SUB_ATT']).fillna(0)
_.drop(['SUBMISSION', 'F_SUB_ATT'], axis=1, inplace=True)
map_dict = _.to_dict('series')['SUBMISSION_ACC']
combatants_df['SUBMISSION_ACC'] = combatants_df['FIGHTER_ID'].map(map_dict).fillna(0)

In [330]:
import numpy as np
opp_sub_defense = bouts_df.loc[:,['OPP_ID', 'SUBMISSION', 'F_SUB_ATT', 'WINNER']]
opp_sub_defense['OPP_SUB_DEFENSE'] = (1 - opp_sub_defense['SUBMISSION'] / opp_sub_defense['F_SUB_ATT'])
opp_sub_defense.loc[opp_sub_defense['OPP_SUB_DEFENSE'] == -np.inf, 'OPP_SUB_DEFENSE'] = 0
opp_sub_defense = opp_sub_defense.dropna().groupby('OPP_ID', as_index=True)['OPP_SUB_DEFENSE'].mean()
combatants_df['SUBMISSION_DEFENSE'] = combatants_df['FIGHTER_ID'].map(opp_sub_defense)

In [343]:
bouts_df['CTRL_RATIO'] = bouts_df['F_CTRL'] / bouts_df['DURATION']
map_dict = bouts_df.groupby('FIGHTER_ID')['CTRL_RATIO'].mean()
combatants_df['CTRL_RATIO'] = combatants_df['FIGHTER_ID'].map(map_dict).fillna(0)

In [13]:
db = DBInterface(cfg.DB_PATH / 'ufc_raw')
df = db.Pdf('fighters')
db.close()
df['Fighter_Name']

dff = df['Fighter_Name'].str.split(' ', expand=True)

dff[dff[4].notnull()]

,0,1,2,3,4
682,Tiago,dos,Santos,e,Silva


In [192]:
db = DBInterface(cfg.DB_PATH / 'ufc_silver')
df = db.Pdf('combatants')
db.close()

df = df.loc[~df['DIVISION_RANK'].isin([None])]
df.sort_values(['GENDER', 'DIVISION', 'DIVISION_RANK'])[['FIGHTER_NAME', 'DIVISION', 'DIVISION_RANK']]

,FIGHTER_NAME,DIVISION,DIVISION_RANK
107,Julia Avila,Bantamweight,11
998,Lina Lansberg,Bantamweight,12
1712,Mayra Bueno Silva,Bantamweight,14
1938,Ketlen Vieira,Bantamweight,2
784,Holly Holm,Bantamweight,3
...,...,...,...
252,Gilbert Burns,Welterweight,5
1857,Stephen Thompson,Welterweight,6
1313,Geoff Neal,Welterweight,7
211,Sean Brady,Welterweight,8
